In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

                                              0.0/232.6 kB ? eta -:--:--
     ------------------                     112.6/232.6 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 3.5 MB/s eta 0:00:00
                                              0.0/10.8 MB ? eta -:--:--
                                              0.1/10.8 MB 2.2 MB/s eta 0:00:05
     -                                        0.3/10.8 MB 3.4 MB/s eta 0:00:04
     ---                                      0.8/10.8 MB 5.9 MB/s eta 0:00:02
     -----                                    1.4/10.8 MB 7.2 MB/s eta 0:00:02
     -------                                  1.9/10.8 MB 8.1 MB/s eta 0:00:02
     ---------                                2.5/10.8 MB 8.8 MB/s eta 0:00:01
     -----------                              3.0/10.8 MB 9.2 MB/s eta 0:00:01
     -------------                            3.6/10.8 MB 9.6 MB/s eta 0:00:01
     ---------------                          4.1/10.8 MB 9.7 M

In [6]:
!pip install pdfreader

                                              0.0/2.9 MB ? eta -:--:--
     -                                        0.1/2.9 MB 2.6 MB/s eta 0:00:02
     ----                                     0.3/2.9 MB 3.5 MB/s eta 0:00:01
     --------                                 0.6/2.9 MB 5.0 MB/s eta 0:00:01
     ----------------                         1.2/2.9 MB 6.8 MB/s eta 0:00:01
     ------------------------                 1.7/2.9 MB 7.8 MB/s eta 0:00:01
     -------------------------------          2.3/2.9 MB 8.5 MB/s eta 0:00:01
     ---------------------------------------  2.8/2.9 MB 9.0 MB/s eta 0:00:01
     ---------------------------------------- 2.9/2.9 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): fin

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [9]:
import pandas as pd
from pdfreader import PDFDocument, SimplePDFViewer
# from text_splitter import CharacterTextSplitter

with open('oCaminhoDosReis.pdf', 'rb') as file:
    reader = PDFDocument(file)

    raw_text = ''
    page_texts = []
    for i, page in enumerate(reader.pages()):
        viewer = SimplePDFViewer(file)
        viewer.navigate(page)
        viewer.render()
        text = "".join(viewer.canvas.strings)
        if text:
            raw_text += text
            page_texts.append((i, text))

    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 1000,
        chunk_overlap  = 200,
        length_function = len,
    )

    df_rows = []
    for page_num, text in page_texts:
        texts = text_splitter.split_text(text)
        for part_num, part in enumerate(texts):
            df_rows.append({'page': page_num, 'part': part_num, 'text': part})

    df = pd.DataFrame(df_rows)

TypeError: unhashable type: 'Page'

In [ ]:
df

In [10]:
reader = PdfReader('oCaminhoDosReis.pdf')

In [11]:
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
print(raw_text)

In [14]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [15]:
len(texts)

2999

In [18]:
texts[567]

'que motivava Adolin. Se conseg uisse derrotar os parshendianos, talvez seu\npai voltasse a ser o homem que já fora. T alvez aqueles delírios espectrais\nque o acossavam desaparecessem.\nÀ fr ente, Dalinar conversava em  voz baixa com Sadeas. Ambos  tinham\no cenho franzido. Eles mal se t oleravam, embora já tivessem sido amigos.\nEra mais uma c oisa que mudara na noite da morte de Gavilar . O que teria\nocorrido entre os dois?\nO dia foi passando e eles final mente chegaram ao local da ca çada —\ndois pla tôs adja centes. Um deles era para onde a criatura seria atraída; o\noutro, a uma dis tância segura, para abrigar os espectadores. Como muitos\nplatôs, aqueles tinham uma superfície acidentada, habitada por plantas\nresistentes, adaptadas a uma exposição contínua às tormentas. Saliências\nrochosas, depressões e chão irregular tornavam traiçoeiras as lutas naquele\nterreno.\nAdolin se juntou ao pai, que estava ao lado da última ponte, esperando'

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
docsearch = FAISS.from_texts(texts, embeddings)

In [28]:
print(docsearch)

# Remember to update and try on GPU

In [22]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [24]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') 

In [23]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [27]:
query = "What is this book about?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O deserto está vivo is a novel about a 14-year-old girl named Gloria Ramírez who escapes her attacker and finds refuge at the door of Mary Rose Whitehead, who is then forced to confront the cruelty and injustices that women in her city have faced for generations. The story is a mix of thriller, western, and mystery genres that explores themes of violence and sisterhood.'